<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/Document_Management/examples_document_person_statement_changed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ased upon: https://github.com/BrainBlend-AI/atomic-agents/blob/main/atomic-examples/web-search-agent/web_search_agent/tools/searxng_search.py

In [ ]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install atomic-agents instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 46.0 MB/s eta 0:00:00


In [ ]:
import instructor
import openai
from openai import OpenAI
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig, BaseIOSchema
from atomic_agents.lib.components.agent_memory import AgentMemory




llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )

client = instructor.from_openai(
    llm,
    mode=instructor.Mode.JSON,
)


In [ ]:
import os
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig, BaseIOSchema
from atomic_agents.lib.components.agent_memory import AgentMemory
from pydantic import Field
from typing import Optional, List
import instructor
from instructor.client import Instructor
from tools.googlesearchtool import GoogleSearchToolOutputSchema


"""
"Analyze the document and identify if the statements made by the person about [specific topic] change or remain consistent throughout the document. Provide examples of any changes or consistencies found."
"""

##################
# Input Schema ###
##################
class DocumentStatementChangedValidatorInputSchema(BaseIOSchema):
    """
    Defines a list of rules and a document.
    """
    document: str = Field(...,description="The document to get analysed.")
    topic: str = Field(...,description="The specific topic we are analsing.")

################
# Output SCHEMA #
################
class DocumentStatementChangedValidatorOutputSchema(BaseIOSchema):
    """
    Schema for Output
    """
    consistent: bool = Field (..., description="Is the statement in the document is consistent or not")
    changes: list[str] = Field (..., description="A list of changes")
    linenumbers: list[int] = Field (..., decription="line numbers where changes of statement occur ")

###################
# Build the agent #
###################

def build_document_statement_changed_validator_agent(client : Instructor, modelid: str) -> BaseAgent:
    """
        builds an agent the validates if a statement has changed over time.

    """
    return BaseAgent(
        config=BaseAgentConfig(
            client=client,
            system_prompt_generator=SystemPromptGenerator(
                background=[
                'You are an expert in legal documents',
                'Your task is to analyse a document'
                ],
                steps=[
                    "Add line numbers to the document."
                    "Analyze the document and identify if the statements made by the person about [specific topic] change or remain consistent throughout the document.",
                    "Provide examples of any changes or consistencies found."
                ],
                output_instructions=[
                    "Ensure to output examples of any changes",
                    "Ensure to add the linenumber where the changes occure."
                ]
            ),
            input_schema=DocumentStatementChangedValidatorInputSchema,
            output_schema=DocumentStatementChangedValidatorOutputSchema,
            model=modelid,
            temperature=0,
        )
    )


"# IDENTITY and PURPOSE\n- This assistant is a general-purpose AI designed to be helpful and friendly.\n\n# INTERNAL ASSISTANT STEPS\n- Understand the user's input and provide a relevant response.\n- Respond to the user.\n\n# OUTPUT INSTRUCTIONS\n- Provide helpful and relevant information to assist the user.\n- Be friendly and respectful in all interactions.\n- Always answer in rhyming verse.\n- Always respond using the proper JSON schema.\n- Always use the available additional information and context to enhance the response."

In [ ]:
document="""
Officer Reynolds: Good afternoon, Mrs. Harper. Thank you for coming in today. I'm Officer Reynolds, and we need to ask you some questions about your husband's death.

Mrs. Harper: Good afternoon, Officer. I'll do my best to help, but I still can't believe what's happened.

Officer Reynolds: I understand. We're just trying to piece together the events of that evening. Can you tell us where you were on the night of the murder?

Mrs. Harper: Yes, I wasn't home that evening. I had a dinner with some friends in town.

Officer Reynolds: You weren’t home at all that night?

Mrs. Harper: No, I came back late, and by the time I got home, it was well past midnight. I found...I found him...

Officer Reynolds: Do you have anyone who can confirm your whereabouts that evening?

Mrs. Harper: Yes, I can provide you with the names of the friends I was with. They can vouch for me.

Officer Reynolds: That would be helpful, Mrs. Harper. Now, if you don’t mind, can you walk me through what happened when you arrived back home?

Mrs. Harper: Of course... When I got back, everything seemed normal at first. Then, I...I saw him lying there...

Officer Reynolds: Take your time. It's important that we get a clear picture of that night.

Mrs. Harper: I understand. I was in shock. I didn't know what to do.

Officer Reynolds: Mrs. Harper, our investigation has led us to believe that you were actually at home during the time of the murder. Are you sure you weren’t there?

Mrs. Harper: Wait, what? No, I told you, I wasn't home.

Officer Reynolds: We have evidence suggesting otherwise. This is your last chance to be honest with us. Were you at home?

Mrs. Harper: (sighs) Okay, okay... I... I was home. But I swear, I was asleep when it happened. I didn't do anything to him!

Officer Reynolds: Thank you for your honesty, Mrs. Harper. We appreciate your cooperation. Can you tell us more about why you initially lied?

Mrs. Harper: I was scared. I didn't want to be blamed for something I didn't do. I thought if I said I wasn’t there, it would be easier. I didn’t know what else to do.

Officer Reynolds: We understand that this is difficult, but telling the truth is always the best course of action. Is there anything else you can share that might help our investigation?

Mrs. Harper: I...I don't know. I just want to find out who did this to him. Please, believe me, I had nothing to do with it.

Officer Reynolds: We will continue our investigation, Mrs. Harper. Thank you for your time. We'll be in touch.

Mrs. Harper: Thank you, Officer. I hope you find the real culprit soon.

Officer Reynolds: We'll do our best, Mrs. Harper. Take care.

"""

In [ ]:
        agent=build_document_statement_changed_validator_agent(llm, args.modelid)


        with open("./data/changing_murder_statement.txt", "r") as file:
            document=file.read()

        input=agent.input_schema(document=document, topic="location at time of murder")
        validationResult=agent.run(input)
        print(validationResult)